In [2]:
!pip install pyspark
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d851c5b39eed832eb48c3abbec8332316a9ed7ad4912651c2f5fb8aec79847ba
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [9]:
import pyspark
from pyspark.sql import SparkSession
from google.colab import files
from pyspark.sql.functions import col
import pandas as pd


In [4]:
spark = SparkSession.builder.appName("OlympicDataAnalysis").getOrCreate()
uploaded = files.upload()


Saving cleaned_olympic_medals_data.csv to cleaned_olympic_medals_data.csv


In [5]:

# Charger le fichier CSV
file_name = list(uploaded.keys())[0]

df = spark.read.csv(file_name, header=True, inferSchema=True)
df.printSchema()

# Filter the data for Paris 1900 Olympics
paris_1900_df = df.filter(df['slug_game'].contains('paris-1900'))
paris_1900_df.show(5)

# Check for the presence of women in the event_gender column
women_participation = paris_1900_df.filter(paris_1900_df['event_gender'].contains('Women')).count()

# result
if women_participation > 0:
    print("Les femmes ont participé aux Jeux Olympiques de Paris en 1900.")
else:
    print("Les femmes n'ont pas participé aux Jeux Olympiques de Paris en 1900.")

root
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- event_gender: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)

+----------------+----------+--------------------+------------+----------+----------------+--------------------+--------------------+------------+---------------------+
|discipline_title| slug_game|         event_title|event_gender|medal_type|participant_type|         athlete_url|   athlete_full_name|country_name|country_3_letter_code|
+----------------+----------+--------------------+------------+----------+----------------+--------------------+--------------------+------------+---------------------+
|        Shooting|paris-1900|arm

In [6]:
df.printSchema()

paris_1900_df = df.filter(df['slug_game'].contains('paris-1900'))
paris_1900_df.show(5)

# List of sports to check
sports_to_check = [
    "natation synchronisée en solo", "tir à la corde", "corde à grimper",
    "montgolfière", "duel au pistolet", "vélo tandem",
    "course d'obstacles à la nage", "plongeon à distance", "tir au pigeon"
]

# Check for the presence of these sports in the event_title column
for sport in sports_to_check:
    sport_presence = paris_1900_df.filter(paris_1900_df['event_title'].contains(sport)).count()
    if sport_presence > 0:
        print(f"Le sport '{sport}' a été disputé aux Jeux Olympiques de Paris en 1900.")
    else:
        print(f"Le sport '{sport}' n'a pas été disputé aux Jeux Olympiques de Paris en 1900.")

root
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- event_gender: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)

+----------------+----------+--------------------+------------+----------+----------------+--------------------+--------------------+------------+---------------------+
|discipline_title| slug_game|         event_title|event_gender|medal_type|participant_type|         athlete_url|   athlete_full_name|country_name|country_3_letter_code|
+----------------+----------+--------------------+------------+----------+----------------+--------------------+--------------------+------------+---------------------+
|        Shooting|paris-1900|arm

In [7]:
df.printSchema()
france_df = df.filter(df['country_3_letter_code'] == 'FRA')
france_df.show(5)

# Group by 'slug_game' to count the number of medals for each edition
medals_by_edition = france_df.groupBy('slug_game').count()
medals_by_edition.show()

# Get the edition with the maximum number of medals
best_result = medals_by_edition.orderBy('count', ascending=False).first()
best_edition = best_result['slug_game']
best_medals_count = best_result['count']

# Get the edition with the minimum number of medals
worst_result = medals_by_edition.orderBy('count', ascending=True).first()
worst_edition = worst_result['slug_game']
worst_medals_count = worst_result['count']

print(f"La France a obtenu son meilleur résultat aux Jeux Olympiques de {best_edition} avec {best_medals_count} médailles.")
print(f"La France a obtenu son pire résultat aux Jeux Olympiques de {worst_edition} avec {worst_medals_count} médailles.")

root
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- event_gender: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)

+----------------+-------------------+--------------------+------------+----------+----------------+--------------------+-----------------+------------+---------------------+
|discipline_title|          slug_game|         event_title|event_gender|medal_type|participant_type|         athlete_url|athlete_full_name|country_name|country_3_letter_code|
+----------------+-------------------+--------------------+------------+----------+----------------+--------------------+-----------------+------------+---------------------+
|         Sail

In [10]:
df.printSchema()
france_df = df.filter(df['country_3_letter_code'] == 'FRA')

# Filter the data for other countries
other_countries_df = df.filter(df['country_3_letter_code'] != 'FRA')

# Group by discipline_title and count the number of medals for France
france_medals_by_discipline = france_df.groupBy('discipline_title').count().withColumnRenamed('count', 'france_medal_count')

# Group by discipline_title and count the number of medals for other countries
other_medals_by_discipline = other_countries_df.groupBy('discipline_title').count().withColumnRenamed('count', 'other_medal_count')

# Join the two DataFrames on discipline_title
comparison_df = france_medals_by_discipline.join(other_medals_by_discipline, 'discipline_title', 'outer').fillna(0)

# Calculate the total medal count and the proportion of medals won by France
comparison_df = comparison_df.withColumn('total_medal_count', col('france_medal_count') + col('other_medal_count'))
comparison_df = comparison_df.withColumn('france_medal_proportion', col('france_medal_count') / col('total_medal_count'))

# Show the disciplines where France has a high proportion of medals
comparison_df.orderBy('france_medal_proportion', ascending=False).show()

root
 |-- discipline_title: string (nullable = true)
 |-- slug_game: string (nullable = true)
 |-- event_title: string (nullable = true)
 |-- event_gender: string (nullable = true)
 |-- medal_type: string (nullable = true)
 |-- participant_type: string (nullable = true)
 |-- athlete_url: string (nullable = true)
 |-- athlete_full_name: string (nullable = true)
 |-- country_name: string (nullable = true)
 |-- country_3_letter_code: string (nullable = true)

+--------------------+------------------+-----------------+-----------------+-----------------------+
|    discipline_title|france_medal_count|other_medal_count|total_medal_count|france_medal_proportion|
+--------------------+------------------+-----------------+-----------------+-----------------------+
|             Croquet|                 8|                0|                8|                    1.0|
|     Military Patrol|                 1|                2|                3|     0.3333333333333333|
|Equestrian  Vaulting|       